In [1]:
import cantera as ct

In [4]:
gas1 = ct.Solution("nasa_gas.yaml")


CanteraError: 
*******************************************************************************
InputFileError thrown by AnyValue::getMapWhere:
Error on line 0 of c:\Users\tahze\AppData\Local\Programs\Python\Python310\lib\site-packages\cantera\data/nasa_gas.yaml:
Key 'phases' not found
|  Line |
|     1 | generator: ck2yaml
|     2 | input-files: [nasa_gas.dat]
|     3 | cantera-version: 2.6.0
*******************************************************************************
